In [73]:
# Standard libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Handeling .wav files

import librosa
from librosa import feature

# Machine Learning

from sklearn.pipeline import Pipeline

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, RepeatedStratifiedKFold

from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from functools import reduce

# data vizualisation

import matplotlib 
import matplotlib.pyplot as plt
import seaborn as sns

In [74]:
def merge_pd(machine):
    
    # Opening the .csv files
    
    df_6dB=pd.read_csv(f'Librosa_features_{machine}_6dB.csv')
    df_0dB=pd.read_csv(f'Librosa_features_{machine}_0dB.csv')
    df_min6dB=pd.read_csv(f'Librosa_features_{machine}_-6dB.csv')
    
    # Dropping the 'Unnamed: 0' column
    
    df_6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_0dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    df_min6dB.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    

    
    # Merging the .csv files into one DataFrame
    
    data_frames = [df_6dB, df_0dB, df_min6dB]
    df_merged = pd.concat(data_frames)
    
    
    return df_merged

In [75]:
fan = merge_pd('fan')

In [76]:
minority_class_len = len(fan[fan['normal(0)/abnormal(1)'] == 0])
minority_class_len

12225

In [77]:
majority_class_len = len(fan[fan['normal(0)/abnormal(1)'] == 1])
majority_class_len

4425

In [78]:
12225/4425

2.76271186440678

In [79]:
X = fan.drop(columns = ['normal(0)/abnormal(1)'])
y = fan['normal(0)/abnormal(1)']

# 20% of the overal data will seperated for later validation of the model

X_model, X_valid, y_model, y_valid = train_test_split(X, y, test_size=0.2, random_state = 42, stratify = y)

# 60% of the overal data (75% of X_model, y_model) will be used to create a training set for the model
# 20% of the overal data (25% of X_model, y_model) will be used to create a testing set for the model

X_train, X_test, y_train, y_test = train_test_split(X_model,
                                                   y_model,
                                                   test_size = 0.25,
                                                   random_state = 10, stratify=y_model)




smote_on_1 = len(fan[fan['normal(0)/abnormal(1)'] == 0]*4

smt = SMOTE(sampling_strategy={1: smote_on_1})
X_train, y_train = smt.fit_sample(X_train, y_train)





sm = SMOTE(random_state = 12)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)


SyntaxError: invalid syntax (<ipython-input-79-46c3168feed0>, line 21)

In [80]:
pipeline = Pipeline([('scaler', StandardScaler()), ('clf', SVC())])

In [81]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 56em; }</style>"))

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.feature_selection import RFE

y_pred = []
clfs = []

clfs.append(DecisionTreeClassifier())
clfs.append(RandomForestClassifier())

#feature_list = [6,13,21]

#for index in feature_list:
for index in range(4,22):
    
    for classifier in clfs:
        
        print("----------------------------------------------")
        print("----------------------------------------------")
        print(classifier)
        print("----------------------------------------------")
        print("----------------------------------------------")
        
        sel = RFE(classifier, n_features_to_select = index)
        sel.fit(X_train, y_train)
        features = X_train.columns[sel.get_support()]
        X_train_rfe = sel.transform(X_train)
        X_test_rfe = sel.transform(X_test)
        print('Selected Feature', index)
        print(features)
    
        classifier.fit(X_train_rfe, y_train)
        y_pred= classifier.predict(X_test_rfe)
        scores = cross_val_score(pipeline, X_train_rfe, y_train, cv=5)


        print("----------------------------------------------")
        print("TRAIN-TEST")
        print("----------------------------------------------")


        print('confusion matrix', classifier)
        print(confusion_matrix(y_test, y_pred))
        print('classification report')
        print(classification_report(y_test, y_pred))
        print('accuracy score')
        print(accuracy_score(y_test, y_pred))
        
        X_valid_rfe = sel.transform(X_valid)
        y_pred = classifier.predict(X_valid_rfe)

        print("----------------------------------------------")
        print("TRAIN-VALIDATION")
        print("----------------------------------------------")

        print('confusion matrix', classifier)
        print(confusion_matrix(y_valid, y_pred))
        print('classification report')
        print(classification_report(y_valid, y_pred))
        print('accuracy score')
        print(accuracy_score(y_valid, y_pred))

----------------------------------------------
----------------------------------------------
DecisionTreeClassifier()
----------------------------------------------
----------------------------------------------
Selected Feature 4
Index(['melspectrogram', 'melspectrogram_std', 'rms', 'spectral_rolloff'], dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2232  213]
 [ 222  663]]
classification report
              precision    recall  f1-score   support

           0       0.91      0.91      0.91      2445
           1       0.76      0.75      0.75       885

    accuracy                           0.87      3330
   macro avg       0.83      0.83      0.83      3330
weighted avg       0.87      0.87      0.87      3330

accuracy score
0.8693693693693694
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion

Selected Feature 7
Index(['melspectrogram', 'melspectrogram_sum', 'melspectrogram_std', 'mfcc',
       'rms', 'spectral_contrast', 'spectral_rolloff'],
      dtype='object')
----------------------------------------------
TRAIN-TEST
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2282  163]
 [ 194  691]]
classification report
              precision    recall  f1-score   support

           0       0.92      0.93      0.93      2445
           1       0.81      0.78      0.79       885

    accuracy                           0.89      3330
   macro avg       0.87      0.86      0.86      3330
weighted avg       0.89      0.89      0.89      3330

accuracy score
0.8927927927927928
----------------------------------------------
TRAIN-VALIDATION
----------------------------------------------
confusion matrix DecisionTreeClassifier()
[[2274  171]
 [ 176  709]]
classification report
              precision    recall  f1-score   support

           0